My Data Cleaning experience:

In [18]:
import yaml
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, inspect
from database_utils import DatabaseConnector
from data_extraction import DataExtractor

yaml_file_path = 'db_creds.yaml'
db_connector = DatabaseConnector(yaml_file_path)

if db_connector.engine:
    tables = db_connector.list_db_tables(db_connector.engine)
    if tables:
        print("Tables in the Database:")
        for table in tables:
            print(table)
    else:
        print("Failed to list database tables.")
else:
    print("Failed to initialize the database engine.")

Tables in the Database:
legacy_store_details
legacy_users
orders_table


Extract the user table

In [19]:
table_name = 'legacy_users'
df = DataExtractor(db_connector).read_rds_table(table_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15320 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.4+ MB


Remove duplicates

In [20]:
df = df[~df.isin(['NULL']).any(axis=1)]

In [21]:
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['phone_number'], keep=False, inplace=True)
df.drop_duplicates(subset=['email_address'], keep=False, inplace=True)
df.drop_duplicates(subset=['user_uuid'], keep=False, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14889 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     14889 non-null  object
 1   last_name      14889 non-null  object
 2   date_of_birth  14889 non-null  object
 3   company        14889 non-null  object
 4   email_address  14889 non-null  object
 5   address        14889 non-null  object
 6   country        14889 non-null  object
 7   country_code   14889 non-null  object
 8   phone_number   14889 non-null  object
 9   join_date      14889 non-null  object
 10  user_uuid      14889 non-null  object
dtypes: object(11)
memory usage: 1.4+ MB


Identify potential category columns

In [25]:
df['country_code'].value_counts()

country_code
GB            8961
DE            4708
US            1205
0CU6LW3NKB       0
PG8MOC0UZI       0
XKI9UXSCZ1       0
VSM4IZ4EL3       0
RVRFD92E48       0
QVUW9JSKY3       0
QREF9WLI2A       0
OS2P9CMHR6       0
44YAIDY048       0
NTCGYW8LVC       0
LZGTB0T5Z7       0
IM8MN1L9MJ       0
FB13AKRI21       0
5D74J6FPFJ       0
XPVCZE2L8B       0
Name: count, dtype: int64

It is clear there are only 3 valid country codes: GB, DE, US. GGB an easily corrected typo.
Viewing the remaining invalid rows shows they can all be removed, also eliminating all 'NULL' values

In [23]:
valid_country_code = ('GB', 'DE', 'US')
df['country'] = df['country'].astype('category')
df['country_code'] = df['country_code'].astype('category')
df['country_code'] = df['country_code'].replace('GGB', 'GB')
incorrect_codes = df[~df['country_code'].isin(valid_country_code)]


In [24]:
df = df.drop(incorrect_codes.index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14874 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   first_name     14874 non-null  object  
 1   last_name      14874 non-null  object  
 2   date_of_birth  14874 non-null  object  
 3   company        14874 non-null  object  
 4   email_address  14874 non-null  object  
 5   address        14874 non-null  object  
 6   country        14874 non-null  category
 7   country_code   14874 non-null  category
 8   phone_number   14874 non-null  object  
 9   join_date      14874 non-null  object  
 10  user_uuid      14874 non-null  object  
dtypes: category(2), object(9)
memory usage: 1.2+ MB


Locate the incorrect date formats that would not be converted with pd.to_datetime

In [26]:
def convert_incorrect_date_formats(column_name):
    condition = ~df[column_name].str.match(r'\d{4}-\d{2}-\d{2}')
    df.loc[condition, column_name] = df.loc[condition, column_name].apply(convert_to_yyyymmdd)
    return df

In [27]:
from datetime import datetime

def convert_to_yyyymmdd(date_str):
    try:
        # Attempt to parse the date using different date formats
        date = datetime.strptime(date_str, '%Y %B %d')
    except ValueError:
        try:
            date = datetime.strptime(date_str, '%B %Y %d')
        except ValueError:
            try:
                date = datetime.strptime(date_str, '%Y/%m/%d')
            except ValueError:
                # If none of the formats match, return None or raise an exception
                return None

    # Convert the date to 'YYYY-MM-DD' format
    yyyymmdd = date.strftime('%Y-%m-%d')
    return yyyymmdd

Convert the incorrect date formats to 'YYYY-MM-DD' format

In [28]:
condition = ~df['date_of_birth'].str.match(r'\d{4}-\d{2}-\d{2}')
print(df.loc[condition, 'date_of_birth'])

index
360       1968 October 16
697            1971/10/23
1631      January 1951 27
1997     November 1958 11
3069      1946 October 18
3616           1974/06/06
3800           2000/01/06
4207     1979 February 01
4594           2003/09/21
5353         June 1943 28
5426     November 1963 06
5535           1998/08/14
6226         July 1966 08
7176           2001/07/28
7267      1948 October 24
8129     December 1946 09
8285           1942/05/29
8536           2001/10/23
9946      2005 January 27
10258        July 1961 14
11218        July 1939 16
13068     1951 January 14
13184          1944/10/15
14128          1981/01/09
14570         May 1996 25
9315           1944/11/30
Name: date_of_birth, dtype: object


In [29]:
# Apply the conversion function to specific rows
df.loc[condition, 'date_of_birth'] = df.loc[condition, 'date_of_birth'].apply(convert_to_yyyymmdd)
print(df.loc[condition, 'date_of_birth'])

index
360      1968-10-16
697      1971-10-23
1631     1951-01-27
1997     1958-11-11
3069     1946-10-18
3616     1974-06-06
3800     2000-01-06
4207     1979-02-01
4594     2003-09-21
5353     1943-06-28
5426     1963-11-06
5535     1998-08-14
6226     1966-07-08
7176     2001-07-28
7267     1948-10-24
8129     1946-12-09
8285     1942-05-29
8536     2001-10-23
9946     2005-01-27
10258    1961-07-14
11218    1939-07-16
13068    1951-01-14
13184    1944-10-15
14128    1981-01-09
14570    1996-05-25
9315     1944-11-30
Name: date_of_birth, dtype: object


In [30]:
condition = ~df['join_date'].str.match(r'\d{4}-\d{2}-\d{2}')
print(df.loc[condition, 'join_date'])
# Apply the conversion function to specific rows
df.loc[condition, 'join_date'] = df.loc[condition, 'join_date'].apply(convert_to_yyyymmdd)

index
1074           2006/10/04
1349      2001 October 14
1571         1998 June 28
2441           2017/01/15
2482      2022 October 04
3167           2008/12/23
4165     2008 December 05
4360     1994 February 12
5557           2008/05/09
6138     November 1994 28
6305     February 2019 03
6494         July 2002 21
8955          May 1999 31
9035          May 1994 27
9368           2019/09/12
10682          2009/06/23
11376          2021/10/09
12163       March 2011 04
12711    December 1992 09
12759          2009/03/05
14335          1997/07/14
14514     October 2022 26
Name: join_date, dtype: object


Convert date columns to datetime objects

In [31]:
date_columns = ('date_of_birth', 'join_date')
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')



In [32]:
display(df['phone_number'])

index
0         +49(0) 047905356
1          (0161) 496 0674
2        +44(0)121 4960340
3          (0306) 999 0871
5        277-664-6389x8405
               ...        
14913      +44(0)292018946
14994     +44(0)1144960977
15012          02984 08192
15269         239.711.3836
1249      +44(0)1314960870
Name: phone_number, Length: 14874, dtype: object

Locate the incorrectly formatted email addresses

In [ ]:
email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+'
invalid_email_df = df[~df['email_address'].str.contains(email_pattern, na=False)]
invalid_email_df

Correct input errors

In [ ]:
# Replace @@ with @ in the "email" column
df['email_address'] = df['email_address'].str.replace('@@', '@')

Clean phone number data

In [ ]:
def format_gb_to_e164(phone_number):
    # Remove non-digit characters
    phone_number = ''.join(filter(str.isdigit, phone_number))
    
    # Check if the first two digits are "44"
    if phone_number[:3] == "440":
        # Phone number has an unecessary 0 at index 2
        return "+" + phone_number[0:2] + phone_number[3:]
    elif phone_number[:3] == "441":
        # Phone number is already in E.164 format
        return "+" + phone_number
    elif phone_number.startswith("0"):
        # Remove the leading "0" and add "44" in front
        return "+44" + phone_number[1:]
    else:
        # Phone number does not start with "44" or "0"
        return None  # Invalid format, you can handle it accordingly
    
def format_de_to_e164(phone_number):
    # Remove non-digit characters
    phone_number = ''.join(filter(str.isdigit, phone_number))
    
    # Check if the first two digits are "49"
    if phone_number[:3] == "490":
        # Phone number has an unecessary 0 at index 2
        return "+" + phone_number[0:2] + phone_number[3:]
    elif phone_number[:3] == "491":
        # Phone number is already in E.164 format
        return "+" + phone_number
    elif phone_number.startswith("0"):
        # Remove the leading "0" and add "49" in front
        return "+49" + phone_number[1:]
    else:
        # Phone number does not start with "49" or "0"
        return None  # Invalid format, you can handle it accordingly    
    


In [ ]:
df.loc[df['country_code']=='GB', 'phone_number'] = df.loc[df['country_code']=='GB']['phone_number'].apply(format_gb_to_e164)

In [ ]:
# Convert to string
df['phone_number'] = df['phone_number'].apply(lambda x: str(x))

In [ ]:
df.loc[df['country_code'] == 'GB', 'phone_number'] = df.loc[df['country_code']=='GB']['phone_number'].apply(lambda x: x[0:3] + ' ' + x[3:])
df.loc[df['country_code'] == 'GB', 'phone_number'] = df.loc[df['country_code']=='GB']['phone_number'].apply(lambda x: x[:-6] + ' ' + x[-6:])

In [ ]:
df.loc[df['country_code'] == 'GB', 'phone_number'] = df.loc[df['country_code']=='GB']['phone_number'].apply(lambda x: x[:-4] + ' ' + x[-4:])
df.loc[df['country_code'] == 'GB', 'phone_number'] = df.loc[df['country_code']=='GB']['phone_number'].apply(lambda x: x[:-8] + ' ' + x[-8:])

In [ ]:
df.loc[df['country_code'] == 'GB', 'phone_number']

In [ ]:
df.loc[df['country_code'] == 'DE', 'phone_number'] = df.loc[df['country_code'] == 'DE']['phone_number'].apply(format_de_to_e164) 

In [ ]:
df.loc[df['country_code'] == 'DE', 'phone_number']

In [ ]:
df.loc[df['country_code'] == 'US', 'phone_number'].tail(20)

In [ ]:
df['address'] = df['address'].str.replace("\n", ' ')
df['address'] = df['address'].str.title()
df['address'] = df['address'].str.split().apply(lambda x: ' '.join(x[:-2] + [word.upper() for word in x[-2:]]))


In [ ]:
# Define a custom function to uppercase the last two words
def uppercase_last_two_words(text):
    words = text.split()
    return ' '.join(words[:-2] + [word.upper() for word in words[-2:]])

# Apply the custom function to the 'address' column based on the 'country_code' condition
df.loc[df['country_code'] == 'GB', 'address'] = df.loc[df['country_code'] == 'GB']['address'].apply(uppercase_last_two_words)



In [ ]:
df.loc[df['country_code'] == 'DE', 'address']

In [ ]:
df.head(20)